<a href="https://colab.research.google.com/github/badro97/NLP_Practice/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
!pip install konlpy

In [ ]:
!pip install soykeyword

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt
from tqdm import tqdm
import re
from soykeyword.proportion import CorpusbasedKeywordExtractor

In [ ]:
df = pd.read_excel('/content/gdrive/MyDrive/싼데비스탄/비버웍스_샘플데이터_open.xlsx', sheet_name=None)
dirty = pd.read_excel('/content/gdrive/MyDrive/싼데비스탄/해커톤참고자료(dirty).xlsx')

In [ ]:
# local
df = pd.read_excel('/Users/badro97/NLP/NLP_Practice/비버웍스_샘플데이터_open.xlsx', sheet_name=None)
dirty = pd.read_excel('/Users/badro97/NLP/NLP_Practice/해커톤참고자료(dirty).xlsx')

In [ ]:
orderQueen = df['오더퀸']
unos = df['우노스']

# 전처리
### 모든 데이터에 적용 가능해야 함
- 동일한 데이터프레임 Form으로 변경
- 취사선택 기능 (제거할 특수문자 입력)
- 동일한 기준 설정 (영어, 단위 제거)
- 오탈자 교정 (자모 단위)
- 띄어쓰기 교정 (soynlp - soyspacing)



**1/16 아이디어**

doc2vec or word2vec kmeans(elbow method)가 줄어들때 까지

태그로 뽑은 단어 제목에 추가(빈도 수 기준(뒷 단어 기준))

스캐터플롯으로 밀집도 변화 관찰

마지막에 메뉴명 최종 전처리(자모 교정, 토큰화, pororo, 외래어 표기법)

In [ ]:
## 데이터프레임 form 통일
def form(df, brand, title, category=''):
  data = pd.DataFrame()
  data['brand'] = df[brand]
  data['title'] = df[title]
  data['category'] = df[category] if category else np.nan
  return data

In [ ]:
o = form(orderQueen, '브랜드명', 'MENU_FULL_NM')
u = form(unos, '매장명', '메뉴명', '카테고리명')
d = form(dirty, '상점명', '상품명', '카테고리명')

df_list = [o, u, d]
df = pd.concat(df_list, ignore_index=True)
df

In [ ]:
df[df['category'].isnull()]

In [ ]:
df['category'].unique()

In [ ]:
df.columns

In [ ]:
## 토큰들 문자열로 결합
def tokenToString(token_list):
    res = []
    for token in token_list:
      s = ' '.join(token)
      res.append(s)
    return res

In [ ]:
## []안에 문자가 있다? (브랜드명), ()안에 한글이 있다? or 숫자뒤에 한글/영어가 온다? (단위)
pat = re.compile("\[[^)]*\]|\([가-힣]\)|[0-9]+[a-zA-Z가-힣]*") 
## 특수문자 제외
sp = re.compile("[^가-힣a-zA-Z\s]")

## 지우는 전처리
def remove_eda(df, col):
  token_list = []
  for t in tqdm(df[col]):
    if t:
      t1 = pat.sub("", str(t))
      text = sp.sub(" ", t1).lower()
      res = text.split()
      token_list.append(res)
    else:
      token_list.append(t)
  result = tokenToString(token_list)
  df[col] = result
  return df

In [ ]:
for col in df.columns:
  df2 = remove_eda(df, col)

In [ ]:
df2

In [ ]:
df2['brand'].unique()

In [ ]:
df2['category'].unique()

In [ ]:
data = df2.copy()

In [ ]:
## 카테고리명split 한 값이 두 개 이상이고 영어가 섞여있다면 한글을 선택

def E_or_K(input):
  k = 0
  e = 0
  for c in input:
    if ord('가') <= ord(c) <= ord('힣'):
      k+=1
    elif ord('a') <= ord(c.lower()) <= ord('z'):
      e+=1
    return "korean" if k > e else "english"

cats = []
for cat in data['category']:
  l = cat.split()
  if len(l) >= 2:
    for text in l:
      ke = E_or_K(text)
      if ke == 'english':
        l.remove(text)
  s = ' '.join(l)
  cats.append(s)

data['category'] = cats

In [ ]:
## 브랜드명 지점위치 제거

brands = []
for brand in data['brand']:
  b = brand.split()
  brands.append(b[0])

data['brand'] = brands

In [ ]:
data['brand'].unique()

In [ ]:
data['category'].unique()

## 카테고리는 메뉴와 가장 가까운 값으로 선택

pororo 제로샷 분류

In [ ]:
!pip install --upgrade pip
!git clone https://github.com/kakaobrain/pororo.git

In [ ]:
cd pororo

In [ ]:
!pip install -e .

In [ ]:
from pororo import Pororo

In [ ]:
# Pororo 제로샷 분류모델
topic_cls = Pororo(task='zero-topic', lang='ko')

In [ ]:
## 카테고리 토큰이 두 개 이상일 경우 메뉴명과 가장 가까운 토큰 선택
def cat_eda(title, cats):
  choice = []
  for i, cats in tqdm(enumerate(cats)):
    cat = cats.split()
    if len(cat) >= 2:
      result = topic_cls(title[i], cat)
      close = max(result, key=(lambda k: result[k]))
      choice.append(close)
    else:
      choice.append(cats)
  return choice

In [ ]:
cat_choice = cat_eda(data['title'], data['category'])
data['category'] = cat_choice

In [ ]:
data['category'].unique()

In [ ]:
## 카테고리가 영어일 경우는 따로 빼놓고 replace로 한글화 해주는 방법이 가장 정신 건강에 좋을 듯 싶다.

In [ ]:
## 키워드 뽑기 (빈도수)
from soykeyword.proportion import CorpusbasedKeywordExtractor
corpusbased_extractor = CorpusbasedKeywordExtractor(
    min_tf=2,
    min_df=1,
    tokenize=lambda x:x.strip().split(),
    verbose=True
)

# docs: list of str like
corpusbased_extractor.train(df2['title'])

In [ ]:
keywords = corpusbased_extractor.extract_from_word(
    '카모마일',
    min_score=0.5,
    min_frequency=0
)

In [ ]:
keywords

## OrderQueen

In [ ]:
orderQueen.info()

In [ ]:
orderQueen.duplicated().sum()

In [ ]:
orderQueen.head()

In [ ]:
orderQueen.tail()

# Unos

In [ ]:
unos.info()

In [ ]:
unos[unos['카테고리명'].isnull()]

In [ ]:
unos['메뉴코드'].duplicated().sum()

In [ ]:
unos.head()

In [ ]:
unos.tail()

# Dirty Sample

In [ ]:
dirty.info()

In [ ]:
dirty